# Introduction 
This notebook and the functions it utilises in the utils.py module and from the dspace module by Lomnitz and Savageau helps you find parameter sets leading to limit cycle oscillations in a kinetic model and automates finding the parameter occurence in those phenotypes that are valid, have oscillatory potential and actually generate limit cycles

## Setup
* Define a model in the model_definitions.py file. This should contain:
    * A Scipy model definition
    * A GMA version of the model for the SDS module
    * Some variables defining the parameters, variables and constraints on the model
    * Choose a set of bifurcation parameters and variables to plot timecourses for. 

## Execution
* Run the notebook below. It will
    * Find valid phenotypes within the model
    * Find all valid phenotypes with potential to oscillate through sampling: 2 posititive real part, complex conjugate eigenvalues
    * Samples each such phenotype with 4 different appoaches
    * Saves any limit cycles it finds
    * For each phenotype with a limit cycle it generates time course plots. 

# Load necessary Python modules

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

SMALL_SIZE = 26
MEDIUM_SIZE = 30
BIGGER_SIZE = 34
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import os
import math
import scipy
from scipy.integrate import odeint
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
import ast
from time import time
import re
from random import shuffle
import pickle
from tqdm import tqdm_notebook

import dspace
import dspace.plotutils

from utils import int_model, draw_single_pheno_samples, check_oscillation, \
                  process_check_plot_pset, count_parameter_occurence, merge_two_dicts, \
                  random_sample_phenotype, update_stability, load_model_variables, build_analyse_design_space

print 'System information:'
print 'dspace            ' + dspace.__version__
print 'libdesignspace    ' + dspace.__c_toolbox__.__version__
print 'matplotlib        ' + matplotlib.__version__
print 'numpy             ' + np.__version__
print 'sciPy             ' + scipy.__version__

# Sampling process

## Looping over models and phenotypes: sampling

In [ ]:
#############################
# Settings
#############################
models_loop_list = ['lomnitz2015'] #['Design 1A',...]
num_iter_per_model = 20

for curr_iter in range(num_iter_per_model):
    time_iter = time()
    
    for model in models_loop_list:
        time_model_iter = time()
        print '### Sampling model', model, '###'
        
        ### load required variables ###
        scipy_model, pset, variables, y0, f, constraints, parbounds, latex_symbols, varnames = load_model_variables(model.lower().replace(' ','_'))

        ### build design space and save characteristics ###
        ds, valid_cases = build_analyse_design_space(model, f, constraints, latex_symbols)

        ### load previous results or start new dataframes ###
        path_to_iterations = '../LCs/'+model+'/total_iter_performed.txt'
        path_to_stability = '../LCs/'+model+'/Stability'
        path_to_samples = '../LCs/'+model+'/LC_parameters'

        if os.path.exists(path_to_stability):
            with open(path_to_iterations,'r') as f: 
                total_iter_performed = int(f.read())

            df_stability = pd.read_pickle(path_to_stability)
            df_LC_samples = pd.read_pickle(path_to_samples) 
        else:
            # start from 0 iterations
            with open(path_to_iterations, 'w') as f:
                total_iter_performed = 0
                f.write(str(total_iter_performed))
                
            ### Build stability dataframe
            d_stability = {}
            for pheno in valid_cases:
                case = ds(pheno)
                d_stability[pheno] = {'Oscillatory potential':False,'Signature':case.signature}
            df_stability = pd.DataFrame.from_dict(d_stability,orient='index')
            df_stability['Complex conjugate sample count'] = 0
            
            ### init a dataframe to store all limit cycle parameter sets
            df_LC_samples = pd.DataFrame(columns=['n','Parameters','IC','Peak sequence','Period','Min. Amplitude (min/max)'])


        print total_iter_performed, "iterations performed so far"
        print "Starting iteration number:",curr_iter + 1

        #############################
        # Sampling and analysis
        #############################
        # save sample stability info
        d_stability = {}
        print "Sampling", len(valid_cases), "valid phenotypes."

        for num_pheno, pheno in enumerate(tqdm_notebook(valid_cases)):            
            case = ds(pheno)

            ### get random parameterset in phenotype 
            pset, IC = random_sample_phenotype(case, parbounds)

            ### analyse stability: look for 2 pos. real part complex conjugates
            ssys = case.ssystem
            alts = ssys.remove_algebraic_constraints()
            ev = alts.eigenvalues(pset)
            pos_roots = sum([ev[i].real >= 0 for i in range(len(ev))])

            if pos_roots == 2:
                complex_pos_roots = [ev[i] for i in range(len(ev)) if (ev[i].real >= 0) and( abs(ev[i].imag) > 0)]

                if len(complex_pos_roots) == 2:
                    # are they conjugate?
                    complex_conjugate = (complex_pos_roots[0].real == complex_pos_roots[1].real) and (complex_pos_roots[0].imag == -1*complex_pos_roots[1].imag)                    
                else:
                    complex_conjugate = False

            else:
                complex_conjugate = False

            ### update df_stability
            if complex_conjugate:
                df_stability.at[pheno,'Oscillatory potential'] = True
                df_stability.at[pheno,'Complex conjugate sample count'] += 1
                    

            ### Check full model for oscillations
            if complex_conjugate:
                phenotype_case = ds(pheno)

                #############################
                # extract subdf for this pheno, indexed by LC number
                #############################
                # identify if we previously found limit cycles for this phenotype
                if pheno in df_LC_samples.index.tolist():
                    subdf_LC_samples_prev = df_LC_samples.loc[[pheno]].set_index('n')
                else:
                    subdf_LC_samples_prev = pd.DataFrame(columns=['n','Parameters','IC','Peak sequence','Period','Min. Amplitude (min/max)']).set_index('n') # empty df

                # analyse full model timecourse for sampled parameterset
                subdf_LC_samples = process_check_plot_pset(scipy_model,phenotype_case,pset,IC,subdf_LC_samples_prev,parbounds,ds,model,varnames)

                # update dataframe if needed
                if len(subdf_LC_samples) > len(subdf_LC_samples_prev):
                    #############################
                    # merge subdf_LC_samples back into df_LC_samples
                    #############################
                    # make indices the same
                    subdf_LC_samples['n'] = subdf_LC_samples.index
                    subdf_LC_samples.index = [pheno]*len(subdf_LC_samples)

                    # Drop old results to avoid duplicates
                    subdf_LC_samples = subdf_LC_samples[subdf_LC_samples['n'] > len(subdf_LC_samples_prev)]

                    df_LC_samples = df_LC_samples.append(subdf_LC_samples)

        #############################
        # save the stability dataframe
        #############################
        # write out updated dataframe of stability
        df_stability = df_stability.sort_values(['Oscillatory potential','Complex conjugate sample count','Signature'], ascending=[False, False, True])
        df_stability.to_pickle(path_to_stability)
        df_stability.to_excel(path_to_stability+'.xlsx')

        # print stability distribution
        print "Currently we identified:", len(df_stability[df_stability['Oscillatory potential']]), "phenotypes with oscillatory potential."

        #############################
        # Parameter occurence for phenotypes with oscillatory potential
        #############################
        # Filter oscillatory potential phenotypes + Parameter occurence
        df_pheno_osc_potential = df_stability[df_stability['Oscillatory potential']]

        parcount = count_parameter_occurence(ds, df_pheno_osc_potential.index.tolist())
        parcount.to_excel('../LCs/'+model+'/Parameter_occurence_phenotypes_oscillatory_potential.xlsx')

        #############################
        # Update and save the limit cycle parameter set dataframe
        #############################
        # sort and save dataframe of limit cycles
        df_LC_samples['Phenotype'] = df_LC_samples.index
        df_LC_samples = df_LC_samples.sort_values(['Phenotype','n'], ascending=True)
        df_LC_samples = df_LC_samples.drop(['Phenotype'],axis=1)

        df_LC_samples.to_excel('../LCs/'+model+'/LC_parameters.xlsx')
        df_LC_samples.to_pickle('../LCs/'+model+'/LC_parameters')
        
        ### LC phenotype parameter occurence
        LC_phenos = list(set(df_LC_samples.index))
        if len(LC_phenos) > 0: 
            print len(LC_phenos)
        else:
            print 'No limit cycles found yet.'

        parcount = count_parameter_occurence(ds, LC_phenos)
        parcount.to_excel('../LCs/'+model+'/Parameter_occurence_phenotypes_LC.xlsx')
        
        ### LC parameter occurence
        LC_phenos = df_LC_samples.index.tolist()
        if len(LC_phenos) > 0: 
            print len(LC_phenos)
        else:
            print 'No limit cycles found yet.'

        parcount = count_parameter_occurence(ds, LC_phenos)
        parcount.to_excel('../LCs/'+model+'/Parameter_occurence_all_LC.xlsx')


        ### increment iterations
        with open(path_to_iterations, 'w') as f:
            total_iter_performed += 1
            f.write(str(total_iter_performed))
            
        ### Print how long this model's iteration took
        print '### This model iteration lasted:', round(time() - time_model_iter,2), 'seconds'
        
    ### Print how long the whole iteration took
    print '### This total iteration lasted:', round(time() - time_iter,2), 'seconds \n'